In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [2]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
file_name="narsingdi_data_with_ipa_ipa"
df = pd.read_excel(f"{file_name}.xlsx")

print(df.head())
print(df.shape)

    District          External_ID  \
0  Narsingdi  rec_23_audio_46.wav   
1  Narsingdi  rec_23_audio_46.wav   
2  Narsingdi  rec_23_audio_46.wav   
3  Narsingdi  rec_23_audio_46.wav   
4  Narsingdi  rec_23_audio_46.wav   

                                            Contents  \
0                    তারা তো ওই খারাপ খেইলাই আসে না।   
1  তারা ভালো, ওই লিগগুলাতে ভালো খেলে, ডোমিস্টিকে ...   
2                  পরেই তো ওদেরকে জাতীয় দলে ডাকা হয়।   
3                                                 হ।   
4                                 কিন্তু, কী করবি ক?   

                                                 ipa  
0          t̪ara t̪o o͡i̯ kʰarap kʰe͡i̯la͡i̯ aʃe na।  
1  t̪ara bʱalo, o͡i̯ liggulat̪e bʱalo kʰele, domi...  
2        pɔre͡i̯ t̪o od̪erke ɟat̪iʲo d̪ɔle daka hɔʲ।  
3                                                ho।  
4                                kint̪u, ki korbi k?  
(9963, 4)


In [3]:
df.head()

,District,External_ID,Contents,ipa
0,Narsingdi,rec_23_audio_46.wav,তারা তো ওই খারাপ খেইলাই আসে না।,t̪ara t̪o o͡i̯ kʰarap kʰe͡i̯la͡i̯ aʃe na।
1,Narsingdi,rec_23_audio_46.wav,"তারা ভালো, ওই লিগগুলাতে ভালো খেলে, ডোমিস্টিকে ...","t̪ara bʱalo, o͡i̯ liggulat̪e bʱalo kʰele, domi..."
2,Narsingdi,rec_23_audio_46.wav,পরেই তো ওদেরকে জাতীয় দলে ডাকা হয়।,pɔre͡i̯ t̪o od̪erke ɟat̪iʲo d̪ɔle daka hɔʲ।
3,Narsingdi,rec_23_audio_46.wav,হ।,ho।
4,Narsingdi,rec_23_audio_46.wav,"কিন্তু, কী করবি ক?","kint̪u, ki korbi k?"


In [11]:
bangla_char_list=['ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১',"২", '৩', '৪', '৫', '৬', '৭', '৮', '৯']
def is_bangla(text):
  for i in text:
    if i in bangla_char_list:
      print(text)
      return True
  return False

In [12]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    # print(f"Started {row.Index}")
    if not is_bangla(row.ipa):
      continue
    print(f"Started {row.Index} {row.Contents}")
    words=row.ipa.split(" ")
    if(len(words)>1):
      datas=""
      for word in words:
        # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
        if not is_bangla(word):
          datas=datas+" "+word
          continue
        url= f"http://127.0.0.1:8001//post_text_bangla_ipa?bangla_ipa={word}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            datas=datas+" "+data["data"][0]
          else:
            datas=datas+" "+"ERROR"
        except:
          print(f"error at{row.Index}:{row.Contents}")
          datas=datas+" "+"ERROR"
      print(f"Ended {row.Index}")
      df.at[row.Index,"ipa"]=datas
    else:
      url = f"http://127.0.0.1:8001//post_text_bangla_ipa?bangla_ipa={row.Contents}"
      response=None
      try:
        response = requests.get (url, verify=False)
        if response and response.status_code == 200:
          data= response.json()
          df.at[row.Index,"ipa"]=data["data"][0]
          print(f"Ended {row.Index}:{row.Contents}")
          print(f"{row.Index}:{data['data'][0]}")
        else:
          data= None
      except:
        print(f"error at{row.Index}:{row.Contents}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [5]:
def subset_worker(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started{row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.sentence}"
    response=None
    try:
      response = requests.get (url, verify=False)
      if response and response.status_code == 200:
        data= response.json()
        df.at[row.Index,"ipa"]=data["data"][0]
      else:
        data= None
    except:
      print(f"error at{row.Index}:{row.sentence}")
    print(f"Ended {row.Index}")
    # print(f"{row.Index}:{row.ipa}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [6]:
def subset_worker_db(subset):
  for row in subset.itertuples():
    data=call_api(row.word)
    if data:
      print(f"{row.Index}:{data['data'][0]}")
    else:
      print(row.Index)
      print(row.word)
      break

In [4]:
# threadpool make sure everything is currect before running this cell

i=0;
i_end=1000
i_increment=200
while(i<=i_end):
  start=i
  steps=20
  end=i+i_increment
  subsets=[]
  start_pos=start
  while start<end:
    subsets.append(df.iloc[start:start+steps])
    start=start+steps
  print(subsets)
  pool=ThreadPoolExecutor(len(subsets))
  ws=[]
  for subset in subsets:
    ws.append(pool.submit(subset_worker_words, subset))
  print(pool.shutdown())
  df.to_csv(f"{file_name}_{start_pos}_{end}.csv",index=False)
  i=i+i_increment

[     District          External_ID  \
0   Narsingdi  rec_23_audio_46.wav   
1   Narsingdi  rec_23_audio_46.wav   
2   Narsingdi  rec_23_audio_46.wav   
3   Narsingdi  rec_23_audio_46.wav   
4   Narsingdi  rec_23_audio_46.wav   
5   Narsingdi  rec_23_audio_46.wav   
6   Narsingdi  rec_23_audio_46.wav   
7   Narsingdi  rec_23_audio_47.wav   
8   Narsingdi  rec_23_audio_47.wav   
9   Narsingdi  rec_23_audio_47.wav   
10  Narsingdi  rec_23_audio_47.wav   
11  Narsingdi  rec_23_audio_47.wav   
12  Narsingdi  rec_23_audio_47.wav   
13  Narsingdi  rec_23_audio_47.wav   
14  Narsingdi  rec_23_audio_48.wav   
15  Narsingdi  rec_23_audio_48.wav   
16  Narsingdi  rec_23_audio_48.wav   
17  Narsingdi  rec_23_audio_48.wav   
18  Narsingdi  rec_23_audio_48.wav   
19  Narsingdi  rec_23_audio_49.wav   

                                             Contents   ipa  
0                     তারা তো ওই খারাপ খেইলাই আসে না।  None  
1   তারা ভালো, ওই লিগগুলাতে ভালো খেলে, ডোমিস্টিকে ...  None  
2             

In [14]:

start=2000
steps=1000
end=10000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_words, subset))
print(pool.shutdown())
df.to_csv(f"{file_name}_{start_pos}_{end}.csv",index=False)

       District               External_ID  \
2000  Narsingdi       rec_26_audio_31.wav   
2001  Narsingdi       rec_26_audio_31.wav   
2002  Narsingdi       rec_26_audio_31.wav   
2003  Narsingdi       rec_26_audio_31.wav   
2004  Narsingdi       rec_26_audio_31.wav   
...         ...                       ...   
2995  Narsingdi       rec_03_audio_39.wav   
2996  Narsingdi       rec_03_audio_39.wav   
2997  Narsingdi  rec_03_small_audio_0.wav   
2998  Narsingdi  rec_03_small_audio_0.wav   
2999  Narsingdi  rec_03_small_audio_0.wav   

                                               Contents  \
2000                       এডি এত রইদে পুড়ার কতা না দো।   
2001  আমার সাইড ইফেক্ট অনেক মানে একদম সাথে-সাথে অইয়া...   
2002   তগো যিমুন ইকটু সময় লাগে আমার সাথে-সাথে হইয়া যায়।   
2003                                    কিয়ের সময় লাগে।   
2004  আমি কইলাম না আমি বাইরে থাইক্কা ঘরো আইয়া দেহি দ...   
...                                                 ...   
2995                                          

In [15]:
df.to_excel(f"{file_name}_withIPA.xlsx",index=False)

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[3000:6000]
for row in subset1_1000.itertuples():
  print(row.Index)
  # print(row[1])
  if pd.notna(row.ipa):
    continue
  print(row.sentence)
  data=call_api(row.sentence)
  # print(data["data"][0])
  if data:
    df.at[row.Index,"ipa"]=data["data"][0]
  else:
    print(row.Index)
    print(row.sentence)
    break
df.to_csv("withIPA.csv",index=False)

**TESTINS**

---



In [3]:
# test connection
def call_api (param):
    url = f"http://127.0.0.1:5000//transcribeBanglaTextToIpa?bangla_ipa={param}"
    response = requests.get (url, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        return None
data=call_api("আমার সোনার বাংলা।")
data["data"][0]

'amar ʃonar baŋla।'

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[20000:70000]
for row in subset1_1000.itertuples():
  # print(row.Index)
  # print(row[1])
  # if pd.notna(row.ipa):
  #   continue
  # print(row.word)
  data=call_api(row.word)
  # print(data["data"][0])
  if data:
    # df.at[row.Index,"ipa"]=data["data"][0]
    print(f"{row.Index}:{data['data'][0]}")
  else:
    print(row.Index)
    print(row.word)
    break
# df.to_csv("withIPA.csv",index=False)

In [ ]:
multiprocessing.cpu_count()

In [43]:
print(pool.shutdown( cancel_futures=True))
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

None


In [ ]:

# w0= pool.submit(subset_worker, subsets[0])
# w1=pool.submit(subset_worker, subsets[1])
# w2=pool.submit(subset_worker, subsets[2])
# w3=pool.submit(subset_worker, subsets[3])
# w4=pool.submit(subset_worker, subsets[4])
# w0.result()
# w1.result()
# w2.result()
# w3.result()
# w4.result()

# print(w0.cancel())
# print(w1.cancel())
# print(w2.cancel())
# print(w3.cancel())
# print(w4.cancel())


In [ ]:
# not working
i=500
steps=100
end=1000
subsets=[]
while i<end:
  subsets.append(df.iloc[i:i+steps])
  i=i+steps
print(subsets)

threads=[]
for subset in subsets:
  threads.append(threading.Thread(target=subset_worker, daemon=True, args=(subset,)))
for thread in threads:
  thread.start();
  time.sleep(1)
for thread in threads:
  thread.join()
print("end")

Cleaning

In [97]:
# Unique word clearing
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
df.drop_duplicates(subset=['bangla_token'], inplace=True)
print(len(df))
df.sort_values('bangla_token').to_csv("bangla_to_ipa_unique_clean.csv",index=False)



58569


In [99]:
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if len(row.bangla_token)>2 and row.bangla_token[-2]=="!":
    # print(row.bangla_token[1:])

    for i in range(1,110,1):
      if df.at[row.Index-i,"bangla_token"]==row.bangla_token[:-2]:
        print(row.Index)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break

df.drop(remove, inplace=True)
print(len(remove))

0


In [113]:
# check whole dataset
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if row.bangla_token[:-1]=="?":
    # if contains in the whole dataset
    for i in range(len(df)):
      if df.at[i,"bangla_token"]==row.bangla_token[:-1]:
        print(row.Index)
        print(i)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break
df.drop(remove, inplace=True)


In [110]:
df.to_csv("bangla_to_ipa_unique_clean2.csv",index=False)

In [83]:
a="bangla_to_ipa_unique_clean"
print(a[1:])

angla_to_ipa_unique_clean
